In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import numpy as np
import tensorboard
import optuna

In [ ]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward):
    """
    Crea e restituisce l'ambiente Ant-v5 dalla libreria Gymnasium.

    Questa funzione istanzia l'ambiente "Ant-v5", uno degli ambienti recenti e ben supportati
    in Gymnasium. I parametri usati sono:
    - reset_noise_scale (0.1): determina la scala del rumore quando l'ambiente viene resettato.
    - render_mode ('None'): indica che non verrà effettuato il rendering durante l'esecuzione.

    Ritorna:
        gym.Env: l'ambiente Ant-v5 inizializzato.
    """
    
    # Ant-v5 è l’ambiente più recente in Gymnasium.
    return gym.make("Ant-v5", 
                    reset_noise_scale=reset_noise_scale, 
                    forward_reward_weight=forward_reward_weight, 
                    ctrl_cost_weight=ctrl_cost_weight, 
                    healthy_reward=healthy_reward, 
                    render_mode='None')

In [ ]:
# Hyperparameter tuning con Optuna

def objective(trial):
    # Definisci lo spazio di ricerca degli iperparametri
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.0, 1.0)
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.0, 1.0)
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.0, 1.0)
    healthy_reward = trial.suggest_float('healthy_reward', 0.0, 1.0)
    
    # Crea l'ambiente con gli iperparametri suggeriti
    env = make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight, healthy_reward)
    env = DummyVecEnv([lambda: env])
    
    # Definisci altri iperparametri del modello
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    n_steps = trial.suggest_int('n_steps', 2048, 8192, step=2048)
    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256])
    gamma = trial.suggest_float('gamma', 0.9, 0.9999)
    
    # Crea il modello PPO
    model = PPO("MlpPolicy", env, learning_rate=learning_rate, n_steps=n_steps, batch_size=batch_size, gamma=gamma, verbose=0)
    
    # Addestra il modello
    model.learn(total_timesteps=10000)
    
    # Valuta il modello
    episodes = 10
    episode_rewards = []
    for episode in range(episodes):
        obs = env.reset()
        done = False
        episode_reward = 0
        while not done:
            action, _states = model.predict(obs)
            obs, reward, done, info = env.step(action)
            episode_reward += reward
        episode_rewards.append(episode_reward)
    
    # Ritorna la ricompensa media come obiettivo
    return np.mean(episode_rewards)

# Crea uno studio Optuna e ottimizza l'obiettivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Stampa i migliori iperparametri trovati
print("Best hyperparameters: ", study.best_params)